# Gate Decomposition

Adapted from [qgopt/docs/gate-decomposition](https://qgopt.readthedocs.io/en/latest/quick_start.html)

It's known that any two-qubits gate can be decomposed as follows:

```text
u1 u2
cnot
u3 u4
cnot
u5 u6
cnot
u7 u8
```

where each line represent one layer of either single-qubit gates or CNOT gates. In this tutorial, we will show how to perform gate such decomposition for any two-qubits gate.

In [ ]:
import numpy as np
import torch

import numqi


In [ ]:
class GateDecompositionModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.manifold = numqi.manifold.SpecialOrthogonal(dim=2, batch_size=8, dtype=torch.complex128)
        self.unitary_dag = None #we will set this in .set_unitary()
        self.cnot = torch.tensor([[1,0,0,0], [0,1,0,0], [0,0,0,1], [0,0,1,0]], dtype=torch.complex128)

    def set_unitary(self, np0):
        assert np0.shape==(4,4)
        assert np.abs(np0@np0.T.conj() - np.eye(4)).max() < 1e-10, 'unitary matrix required'
        self.unitary_dag = torch.tensor(np0.T.conj().copy(), dtype=torch.complex128)

    def get_unitary(self, return_numpy=True):
        ulist = self.manifold()
        matU = torch.kron(ulist[0], ulist[1])
        for i in range(3):
            matU = self.cnot @ matU
            matU = torch.kron(ulist[2*i+2], ulist[2*i+3]) @ matU
        ret = ulist, matU
        if return_numpy:
            ret = ret[0].detach().numpy(), ret[1].detach().numpy()
        return ret

    def forward(self):
        _,matU = self.get_unitary(return_numpy=False)
        tmp0 = torch.trace(self.unitary_dag @ matU)
        # unitary infidelity https://docs.q-ctrl.com/references/qctrl/Graphs/Graph/unitary_infidelity.html
        loss = 1 - (tmp0.real**2 + tmp0.imag**2) / 16
        return loss


Different from QGopt, we choose unitary infidelity as loss function [qctrl/unitary_infidelity](https://docs.q-ctrl.com/references/qctrl/Graphs/Graph/unitary_infidelity.html)

$$ \mathcal{L}=1 - \lvert\mathrm{Tr}[V(\theta) U^\dagger]\rvert^2 / d^2 $$

for that we ignore the global phase of the target unitary $V$. Above $d$ denotes the dimension of the Hilbert space $d=4$, $U$ for the target untiary matrix and $V(\theta)$ is the parametrized unitary matrix composed of single-qubit gates and CNOT gates.

In [ ]:
target_unitary = numqi.random.rand_special_orthogonal_matrix(4, tag_complex=True)
print(f'target unitary:\n{np.around(target_unitary, 3)}')

tmp0 = target_unitary @ target_unitary.T.conj()
print(f'unitary check (U U^dag): \n{np.around(tmp0, 3)}')


In [ ]:
model = GateDecompositionModel()
model.set_unitary(target_unitary)
theta_optim = numqi.optimize.minimize(model, theta0='uniform', num_repeat=1, print_freq=5, tol=1e-10)


In [ ]:
ulist, matU = model.get_unitary()

print('first several single-qubit unitary matrices:')
for i in range(3):
    print(f'U{i} = \n{np.around(ulist[i], 3)}')


In [ ]:
print(f'final 2-qubit unitary matrix:\n{np.around(matU, 3)}')

tmp0 = matU @ target_unitary.T.conj()
print(f'check unitary (V U^dag):\n{np.around(tmp0,5)}')
